# Human Silhouette & Coordinate-based Tracking
- Sillhouette can be manually resized 
- To make sure silhouette fits within the screen, be sure to modify values<br> **move_x_by, move_y_by** too 
- Optical flow is given manual coordinates for hand

In [1]:
import cv2
import numpy as np

In [2]:
def show_img(img):
    cv2.imshow('img', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

Get image and generate inverted mask of the image

In [3]:
import os
os.getcwd()

'c:\\cv\\opencv\\Scripts\\local_repo\\2017-2-CSP-MuJe-6\\Won-Joon'

### Load image & Create mask

In [4]:
# Read images
img1 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette.png")
# img2 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette (480 x 640).png") # not used!


# Frame is 640 x 480, resize img by 1/3
#resize_factor = 1.8
img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2gray = cv2.resize(img2gray, (int(480/1.8), int(640/1.4)))



# create mask from silhoutte
ret, mask = cv2.threshold(img2gray, 230, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# show_img(img2gray)
# show_img(mask_inv)

### Sparse Optical Flow settings

In [6]:
# Lucas Kanade optical flow parameters
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Random colors
color = np.random.randint(0,255,(100,3))

### Take first frame & assign features

In [7]:
def make_features_xy(x, x2, y, y2):
    return [ [list(item)] for item in zip(list(range(x, x2)), list(range(y, y2)))]

In [8]:
test_list = make_features_xy(100, 120, 100, 120)

In [9]:
test_list

[[[100, 100]],
 [[101, 101]],
 [[102, 102]],
 [[103, 103]],
 [[104, 104]],
 [[105, 105]],
 [[106, 106]],
 [[107, 107]],
 [[108, 108]],
 [[109, 109]],
 [[110, 110]],
 [[111, 111]],
 [[112, 112]],
 [[113, 113]],
 [[114, 114]],
 [[115, 115]],
 [[116, 116]],
 [[117, 117]],
 [[118, 118]],
 [[119, 119]]]

Spare code

In [12]:
# Parameters for displaying silhouette 
y, x = img2gray.shape
move_y_by = 20
move_x_by = 200

# Take first frame and find corners in it
cap = cv2.VideoCapture(0)

ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)
p0 = np.array(   [[[100, 100]]]  , dtype = 'float32' )
print(p0.shape)

# Create a mask image for drawing purposes
mask = np.zeros_like(old_frame)


while ( cap.isOpened() ):
    _, frame = cap.read()    
    
    # Part 1:calculate optical flow ##############################
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    p1, st, err = cv2.calcOpticalFlowPyrLK(prevImg = old_gray,
                                           nextImg = frame_gray, 
                                           prevPts = p0, 
                                           nextPts = None, **lk_params)
    print("p1:", p1, "st:", st, "err:", err)
    # Select good points    
    good_new = p1#[st==1]
    good_old = p0#[st==1]

    # draw the tracks
    for i,(new,old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    
    
     # Part 2: Put silhouette on ROI ###########################
    roi = frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ]
    bg_roi_withMask = cv2.bitwise_and(roi, roi, mask = mask_inv)
    
    destination = cv2.add( frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] , bg_roi_withMask)
    frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] = destination


    
    
    cv2.imshow('feed', frame)
    ch = 0xFF & cv2.waitKey(1)
    if ch == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        

(1, 1, 2)
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st

p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] err: [[ nan]]
p1: [[[ 100.  100.]]] st: [[0]] er

In [33]:
cap.release()

In [56]:
print 'tasks done, now sleeping for 10 seconds'
for i in xrange(10,0,-1):
    time.sleep(1)
    print i

hi


<Timer(Thread-7, initial)>

In [70]:
import cv2
import numpy as np 

# Read images
img1 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette.png")
# img2 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette (480 x 640).png") # not used!

# Frame is 640 x 480, resize img by 1/3
#resize_factor = 1.8
img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
img2gray = cv2.resize(img2gray, (int(480/1.8), int(640/1.4)))

# create mask from silhoutte
ret, mask = cv2.threshold(img2gray, 230, 255, cv2.THRESH_BINARY)
mask_inv = cv2.bitwise_not(mask)

# Parameters for displaying silhouette 
y, x = img2gray.shape
move_y_by = 20
move_x_by = 200


timer = countdown(5)


cap = cv2.VideoCapture(0)

while ( cap.isOpened() ):
    _, frame = cap.read()

    # Put silhouette on ROI
    roi = frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ]
    bg_roi_withMask = cv2.bitwise_and(roi, roi, mask = mask_inv)
    
    destination = cv2.add( frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] , bg_roi_withMask)
    frame[ move_y_by : y + move_y_by , move_x_by : x + move_x_by ] = destination

    cv2.imshow('feed', frame)
    
    if timer == 0:
        break
    
    ch = 0xFF & cv2.waitKey(1)
    if ch == 27:
        cap.release()
        cv2.destroyAllWindows()
        break
        

Duration : 1s

KeyboardInterrupt: 

In [68]:
import sys
def countdown(t):
    while t > 0:
        sys.stdout.write('\rDuration : {}s'.format(t))
        t -= 1
        sys.stdout.flush()
        time.sleep(1)
        if t == 0:
            return 0

In [73]:
import numpy as np
import cv2

# ShiTomasi corner detection parameters
feature_params = dict( maxCorners = 100,
                       qualityLevel = 0.3,
                       minDistance = 7,
                       blockSize = 7 )
# Lucas Kanade optical flow parameters
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))
# Random colors
color = np.random.randint(0,255,(100,3))








if __name__ == '__main__':

  # Read images
    img1 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette.png")
    # img2 = cv2.imread(os.getcwd()+"\\Images\\"+ "Silhouette (480 x 640).png") # not used!

    # Frame is 640 x 480, resize img by 1/3
    #resize_factor = 1.8
    img2gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2gray = cv2.resize(img2gray, (int(480/1.8), int(640/1.4)))

    # create mask from silhoutte
    ret, mask = cv2.threshold(img2gray, 230, 255, cv2.THRESH_BINARY)
    mask_inv = cv2.bitwise_not(mask)

    # Parameters for displaying silhouette 
    y, x = img2gray.shape
    move_y_by = 20
    move_x_by = 200

    ##Optical Flow 삽입
    cap = cv2.VideoCapture(0)

    # Take first frame and find corners in it
    ret, old_frame = cap.read()
    print(old_frame)
    old_gray = cv2.cvtColor(old_frame, cv2.COLOR_BGR2GRAY)

    # Set tracking points 
    p0 = np.array([[[200.0,200.0]]]) 
    p0 = np.float32(p0)

    # Create a mask image for drawing purposes
    mask = np.zeros_like(old_frame)

    while(1):

        ret,realframe = cap.read()
         
        ##############Background UI 삽입################
        backframe = realframe
        roi = backframe[0:rows, 0:cols]

        # create mask from logo 
        img2gray = cv2.cvtColor(img2,cv2.COLOR_BGR2GRAY)
        ret, backmask = cv2.threshold(img2gray, 5, 255, cv2.THRESH_BINARY)
        mask_inv = cv2.bitwise_not(backmask)

        # black out the area of logo in ROI
        img1_bg = cv2.bitwise_and(roi,roi,mask = mask_inv)

        # Take only region of logo from logo image.
        img2_fg = cv2.bitwise_and(img2,img2,mask = backmask)

        # Put logo in ROI and modify the main image
        dst = cv2.add(img1_bg,img2_fg)
        backframe[0:rows, 0:cols ] = dst

        #cv2.imshow('feed',frame)

        ##############################################

        #trackingframe = realframe
        frame_gray = cv2.cvtColor(realframe, cv2.COLOR_BGR2GRAY)

        # calculate optical flow
        p1, st, err = cv2.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)
        # Select good points
        good_new = p1[st==1]
        good_old = p0[st==1]

        # draw the tracks
        for i,(new,old) in enumerate(zip(good_new,good_old)):
            a,b = new.ravel()
            c,d = old.ravel()
            mask = cv2.line(mask, (a,b),(c,d), color[i].tolist(), 2)
            realframe = cv2.circle(realframe,(a,b),5,color[i].tolist(),-1)

        img = cv2.add(realframe,mask)
        
        if a== 60 and b==60:
            break
        
        #img = cv2.add(realframe,backframe)
        #img = cv2.add(realframe,trackingframe)
        
        cv2.imshow('frame',img)
        
        #카메라 종료
        k = cv2.waitKey(30) & 0xff
        if k == 27: #esc key
            cap.release()
            break

        # Now update the previous frame and previous points
        old_gray = frame_gray.copy()
        p0 = good_new.reshape(-1,1,2)
        
    cv2.destroyAllWindows()

[[[5 5 5]
  [5 5 5]
  [5 5 5]
  ..., 
  [6 6 6]
  [6 6 6]
  [6 6 6]]

 [[6 6 6]
  [6 6 6]
  [5 5 5]
  ..., 
  [6 6 6]
  [6 6 6]
  [6 6 6]]

 [[6 6 6]
  [6 6 6]
  [6 6 6]
  ..., 
  [6 6 6]
  [6 6 6]
  [6 6 6]]

 ..., 
 [[5 5 5]
  [5 5 5]
  [5 5 5]
  ..., 
  [4 4 4]
  [4 4 4]
  [4 4 4]]

 [[5 5 5]
  [6 6 6]
  [5 5 5]
  ..., 
  [5 5 5]
  [5 5 5]
  [4 4 4]]

 [[6 6 6]
  [6 6 6]
  [5 5 5]
  ..., 
  [5 5 5]
  [5 5 5]
  [4 4 4]]]


NameError: name 'rows' is not defined

In [74]:
cap.release()